In [ ]:
#데이터 불러오기
import pandas as pd

url = 'https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv'
df = pd.read_csv(url)

df.head()

In [ ]:
#데이터 타입 확인
df.dtypes

In [ ]:
#데이터 전처리
df = df.drop('Unnamed: 0', axis=1)
df['Year'] = df['Year'].astype(str)
df = df.rename(columns = {'Name': '게임명 (Name)','Platform': '지원 플랫폼 (Platform)','Year': '출시 연도 (Year)','Genre': '장르 (Genre)',
                          'Publisher': '제작사 (Publisher)','NA_Sales': '북미 매출액 (NA_Sales)','EU_Sales': '유럽 매출액 (EU_Sales)',
                          'JP_Sales': '일본 매출액 (JP_Sales)','Other_Sales': '기타지역 매출액 (Other_Sales)'
                          })
df.head()

In [ ]:
# 매출액 데이터 전처리 (단위 : 백만/1,000,000)

def sales(a):
  K = pd.to_numeric(df[df[a].str.contains('K')][a].str.split('K').str[0])/1000
  df.loc[K.index,a] = K.astype('string')
  M = pd.to_numeric(df[df[a].str.contains('M')][a].str.split('M').str[0])
  df.loc[M.index,a] = M.astype('string')
  df[a] = pd.to_numeric(df[a])

sales('북미 매출액 (NA_Sales)')
sales('유럽 매출액 (EU_Sales)')
sales('일본 매출액 (JP_Sales)')
sales('기타지역 매출액 (Other_Sales)')

print('$n',df.dtypes)
print(df.isnull().sum())
# print(df.dtypes)

In [ ]:
#결측치 데이터 전처리
df.dropna(subset=['출시 연도 (Year)', '장르 (Genre)', '제작사 (Publisher)'], inplace = True)
df.head()

In [ ]:
#장르별 총 매출액 확인
grouped = df.groupby('장르 (Genre)').sum()
grouped['총 매출액 (Total Sales)'] = grouped['북미 매출액 (NA_Sales)'] + grouped['유럽 매출액 (EU_Sales)'] + grouped['일본 매출액 (JP_Sales)'] + grouped['기타지역 매출액 (Other_Sales)']
grouped.sort_values('총 매출액 (Total Sales)', ascending=False)

In [ ]:
#시각화 한국어 오류 처리
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
#장르별 매출액 시각화

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 10.
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.labelsize'] = 15.

def genre(name,data_name):
  name = pd.DataFrame(df.groupby(['장르 (Genre)'])[data_name].sum()).head(5)
  name.reset_index(inplace=True)
  name=name.sort_values(by=data_name, ascending=False)
  sns.barplot(x=data_name, y='장르 (Genre)', data=name, palette="copper").set_title("지역별 Top 5 선호 게임 장르 (단위: 백만불)")
  print(df.groupby(['장르 (Genre)'])[data_name]).sum().head(5)

print(genre('장르 (Genre)','북미 매출액 (NA_Sales)'))

In [ ]:
print(genre('장르 (Genre)','유럽 매출액 (EU_Sales)'))

In [ ]:
print(genre('장르 (Genre)','일본 매출액 (JP_Sales)'))

In [ ]:
print(genre('장르 (Genre)','기타지역 매출액 (Other_Sales)'))

In [ ]:
#연도별 트렌드 10년 단위
df['연도별 트렌드(10년)'] = df['출시 연도 (Year)'].str[:3]
df_graph = df.groupby(['연도별 트렌드(10년)','장르 (Genre)']).size().reset_index().pivot(columns='장르 (Genre)', index='연도별 트렌드(10년)', values=0)
df_graph

In [ ]:
#연도별 트렌드 데이터셋
df_trend = pd.DataFrame({
              'Action': [66.0, 158.0, 1573.0, 1429.0],
              'Adventure': [2.0, 96.0, 629.0, 537.0],
              'Fighting': [4.0, 191.0, 440.0, 194.0],
              'Misc': [8.0, 115.0, 991.0, 557.0],
              'Platform': [32.0, 124.0, 559.0, 150.0],
              'Puzzle': [19.0, 71.0, 356.0, 112.0],
              'Racing': [8.0, 179.0, 791.0, 231.0],
              'Role-Playing': [9.0, 171.0, 728.0, 549.0],
              'Shooter': [30.0, 136.0, 712.0, 390.0], 
              'Simulation': [3.0, 86.0, 547.0, 204.0],
              'Sports': [22.0, 303.0, 1397.0, 567.0],
              'Strategy': [0.0, 121.0, 377.0, 166.0]},
             index=['1980년', '1990년', '2000년', '2010년'])
df_trend

In [ ]:
#연도별 게임 트렌드 시각화
df_trend.plot().set_title("연도별 게임 트렌드 (단위: 백만불)")

In [ ]:
#매출이 높은 게임 분석
df['총매출 (Total_Sales)'] = df['북미 매출액 (NA_Sales)'] + df['유럽 매출액 (EU_Sales)'] + df['일본 매출액 (JP_Sales)'] + df['기타지역 매출액 (Other_Sales)']
df.sort_values(by='총매출 (Total_Sales)', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(30)

In [ ]:
df['총매출 (Total_Sales)'].sum()